# Animate GPX tracks on the map

The goal is to help aligning a raster orienteering map with GPS, align GPX tracks temporally and display them as a head-to-head race.

In [1]:
from time import sleep
from itertools import tee
from IPython.display import display, FileLink
import ipywidgets as widgets

import sys
sys.path.append("./src")
from tracks import Tracks
from imgmap import ImgMap
from anchors import Anchors
from finalmap import FinalMap

In [2]:
class App:
    def __init__(self):
        self.tracks = None
        self.img_map = None
        self.anchors = None
        self.pictures = []
        self.final_map = None
        self.anchors_out = widgets.Output() #layout={'border': '1px solid black'})
        self.map_out = widgets.Output() #layout={'border': '1px solid black'})

    def get_download_btn(self):
        download_btn = widgets.Button(description="Get the link")
        self.file_link = None
        def on_download_btn_clicked(b):
            if self.file_link:
                return
            self.final_map.m.save('map.html')
            with self.map_out:
                self.file_link = FileLink('map.html')
                display(self.file_link)
        download_btn.on_click(on_download_btn_clicked)
        return download_btn
        
    def redraw_map_out(self):
        self.map_out.clear_output()
        self.final_map = FinalMap(self.tracks, self.anchors, self.pictures)
        download_btn = self.get_download_btn()
        with self.map_out:
            display(self.final_map.m, download_btn)
        
    def set_tracks(self, files):
        self.tracks = Tracks(files)
        self.redraw_map_out()

    def set_image(self, img):
        self.img_map = ImgMap(img, self.tracks)
        self.anchors = Anchors(self.img_map, self.tracks)
        
        plot_btn = widgets.Button(description="Plot the map")
        def on_plot_btn_clicked(b):
            self.redraw_map_out()
        plot_btn.on_click(on_plot_btn_clicked)

        self.anchors_out.clear_output()
        with self.anchors_out:
            display(self.anchors.m, plot_btn)

    def set_pictures(self, pics):
        self.pictures = pics
        self.redraw_map_out()

app = App()

In [3]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

## 1. Select a couple of GPX tracks

The working are will be determined from the first track.

In [4]:
gpx_upload = widgets.FileUpload(accept='.gpx', multiple=True)
display(gpx_upload)

#@debounce(0.2)
def on_tracks_value_change(change):
    app.set_tracks([file['content'] for file in gpx_upload.value.values()])

gpx_upload.observe(on_tracks_value_change, names='value')

FileUpload(value={}, accept='.gpx', description='Upload', multiple=True)

## 2. Now upload a raster image of the orienteering map


Select features on the image as far from each other as possible.
Then mark the same features on the map.

If this step is omitted, there will be no overlay.

In [5]:
img_upload = widgets.FileUpload(accept='image/*', multiple=False)
display(img_upload, app.anchors_out)

#@debounce(0.2)
def on_img_value_change(change):
    img = next(iter(img_upload.value.values()), None)
    if img:
        app.set_image(img['content'])

img_upload.observe(on_img_value_change, names='value')

FileUpload(value={}, accept='image/*', description='Upload')

Output()

## 3. Upload some positional images

In [6]:
pics_upload = widgets.FileUpload(accept='image/*', multiple=True)
display(pics_upload)

def on_pics_value_change(change):
    app.set_pictures([i['content'] for i in pics_upload.value.values()])

pics_upload.observe(on_pics_value_change, names='value')

FileUpload(value={}, accept='image/*', description='Upload', multiple=True)

## 4. Get the map with the aligned image map and track race

In [7]:
display(app.map_out)

Output()